In [1]:
import random, operator, subprocess
from pyspark.sql.types import *
# from pyspark import SparkContext
# sc =SparkContext()

In [2]:
rdd = sc.textFile('data.csv') \
        .map(lambda line: line.split(',')) \
        .map(lambda elements: tuple([int(elements[i]) for i in range(len(elements))])) \
        .cache()

k = 10
dimension = 3
headers = ['age', 'height', 'weight', 'blood_sugar_level', 'child', 'exercise_hours']
# max_cluster = rdd.count() / k
# min_cluster = rdd.count() / (2*k-1)
loop_for_converge = 20
different_combination = 30

In [3]:
def dist(x, y):
    return sum([abs(x[i]-y[i]) for i in range(dimension)])

def get_nearest_centroid_idx(x, centroids):
    dists = {}
    for cluster in centroids:
        dists[cluster] = dist(x, centroids[cluster])
        
    cluster = min(dists, key=dists.get)
    return cluster

def assign_to_cluster(pt, available_centroids):
    nearest_centroid = get_nearest_centroid_idx(pt, available_centroids)
    return (nearest_centroid, ([pt], [dist(pt, available_centroids[nearest_centroid])]))

def calculate_pts_sum(pts):
    pts_sum = [0 for _ in range(dimension)]
    for pt in pts:
        for i in range(dimension):
            pts_sum[i] += pt[i]
    return pts_sum

def calculate_centroid(pts_sum, nb_pts):
    nb_pts = float(nb_pts)
    return [pts_sum[i]/nb_pts for i in range(dimension)]

def calculate_cost(pts, centroid):
    cost = 0
    for pt in pts:
        cost += dist(pt, centroid)
    return cost

def is_converge(old_cens, new_cens):
    diff = 0
    old_sum = 0
    for i in range(dimension):
        old_cen = old_cens[i]
        new_cen = new_cens[i]
        for j in range(dimension):
            diff += abs(new_cen[j] - old_cen[j])
            old_sum += old_cen[j]
    return abs(float(diff) / old_sum) < 0.000001

def write_to_output(assignment, centroids):
    tmp = assignment.flatMap(lambda (cluster, pts): [centroids[cluster] for _ in range(len(pts))])
    sqlContext.createDataFrame(tmp, headers[:dimension]).save('output.txt', mode='overwrite')

In [4]:
min_cost_rdd = None
min_cost = float('inf')

In [29]:
eps = 2
minPts = k
visited = []

In [6]:
def smaller(pt1,pt2):
    for i in range(dimension):
        if pt1[i]<pt2[i]:
            return True
    if pt1==pt2:
        return True
    return False

In [30]:
from collections import deque

def fromCluster(pt1):
    print "from cluster"
    clusterSet=[]
    print pt1
    if (pt1 not in visited):
        if (pt1 in ptsFullNeighborDict):
            visited += pt1
            queue = deque(ptsFullNeighborDict[pt1]) # ptsFullNeighborDict include itself #still a list
            print queue
            while (not queue.empty()):
                nextpt = queue.dequeue()
                if (nextpt in visited): 
                    # TODO:
                    # need to merge cluster since it is parallel
                    # may fail to achieve k-anonymity if don't merge
                    # neighbor of neighbor -> can ignore
                    pass
                else:
                    visited += nextpt
                    clusterSet += nextpt
                    if (nextpt in ptsFullNeighborDict):
                        for nn_pt in ptsFullNeighborDict[nextpt]:
                            queue.push(nn_pt)
    return clusterSet # still not mark as visit, may be visited by other point

In [7]:
# add label
rdd.take(3)
ptsFullNeighborRDD=rdd.cartesian(rdd)\
                        .filter(lambda (pt1,pt2): dist(pt1,pt2)<eps)\
                        .map(lambda (pt1,pt2):(pt1,[pt2]))\
                        .filter(lambda (pt, pts): len(pts)>=minPts)\
                        .reduceByKey(lambda pts1,pts2: pts1+pts2)#(pt1,[pt2,pt3])
# ptsFullNeighborDict = ptsLargerNeighborRDD.collectAsMap()

edgeRDD=rdd.cartesian(rdd)\
                        .filter(lambda (pt1,pt2): dist(pt1,pt2)<eps)\
                        .map(lambda (pt1,pt2):(pt1,[pt2]))\
                        .reduceByKey(lambda pts1,pts2: pts1+pts2)
                
ptsLargerNeighborRDD=rdd.cartesian(rdd)\
                        .filter(lambda (pt1,pt2): smaller(pt1,pt2))\
                        .filter(lambda (pt1,pt2): dist(pt1,pt2)<eps)\
                        .map(lambda (pt1,pt2):(pt1,[pt2]))\
                        .reduceByKey(lambda pts1,pts2: pts1+pts2)\
                        .cache()#(pt1,[pt2,pt3])
                    
#neighbor include itself
#different pt at same location(have same content) can be ignored (since its largerNeighbor must same, it is covered)

In [8]:
print ptsFullNeighborRDD.take(3)
print ptsLargerNeighborRDD.take(3)

[((24, 184, 5), [(24, 184, 5)]), ((94, 176, 1), [(94, 176, 1), (94, 176, 1), (94, 176, 2), (94, 176, 1), (94, 176, 1), (94, 176, 2)]), ((48, 174, 2), [(48, 174, 2)])]
[((24, 184, 5), [(24, 184, 5)]), ((94, 176, 1), [(94, 176, 1), (94, 176, 1), (94, 176, 2), (94, 176, 1), (94, 176, 1), (94, 176, 2)]), ((48, 174, 2), [(48, 174, 2)])]


In [13]:
from graphframes import *
# rdd to dataframe
localVertices = [(1,"A"), (2,"B"), (3, "C")]
localEdges = [(1,2,"love"), (2,1,"hate"), (2,3,"follow")]
v = sqlContext.createDataFrame(localVertices, ["id", "name"])
e = sqlContext.createDataFrame(localEdges, ["src", "dst", "action"])
#vertics = rdd.toDF()
#edges = shortEdge.toDF()
g = GraphFrame(v, e)
sc.setCheckpointDir("/checkpoint") # required for connectedComponents version > 0.3
result = g.connectedComponents().setAlgorithm("graphx")# revier to older version
# .setAlgorithm( "graphx" ) to avoid 
result.select("id", "component").orderBy("component").show()

Py4JJavaError: An error occurred while calling o494.setCheckpointDir.
: java.io.FileNotFoundException: File /checkpoint/ab154a06-8c82-4a40-9795-a2b49dd70661 does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:534)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:747)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:524)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:409)
	at org.apache.spark.SparkContext$$anonfun$setCheckpointDir$2.apply(SparkContext.scala:2078)
	at org.apache.spark.SparkContext$$anonfun$setCheckpointDir$2.apply(SparkContext.scala:2074)
	at scala.Option.map(Option.scala:145)
	at org.apache.spark.SparkContext.setCheckpointDir(SparkContext.scala:2074)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.reflect

In [3]:
from graphframes.example import Graphs
g = Graphs(sqlContext).friends()  # Get example graph

result = g.connectedComponents()
result.select("id", "component").orderBy("component").show()
# rdd.map(lambda pt: fromCluster(pt)).take(3)

ImportError: No module named example

In [2]:
# Create a Vertex DataFrame with unique ID column "id"
v = sqlContext.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
], ["id", "name", "age"])
# Create an Edge DataFrame with "src" and "dst" columns
e = sqlContext.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
], ["src", "dst", "relationship"])
# Create a GraphFrame
from graphframes import *
g = GraphFrame(v, e)

# Query: Get in-degree of each vertex.
g.inDegrees.show()

# Query: Count the number of "follow" connections in the graph.
g.edges.filter("relationship = 'follow'").count()

# Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").show()

+---+--------+
| id|inDegree|
+---+--------+
|  b|       2|
|  c|       1|
+---+--------+



Py4JJavaError: An error occurred while calling o92.run.
: java.lang.NoSuchMethodError: scala.reflect.api.JavaUniverse.runtimeMirror(Ljava/lang/ClassLoader;)Lscala/reflect/api/JavaUniverse$JavaMirror;
	at org.graphframes.lib.GraphXConversions$.fromGraphX(GraphXConversions.scala:68)
	at org.graphframes.lib.PageRank$.run(PageRank.scala:132)
	at org.graphframes.lib.PageRank.run(PageRank.scala:104)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:606)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:381)
	at py4j.Gateway.invoke(Gateway.java:259)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:133)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at 